In [1]:
##Dependencies changed as per instructor permission

!wget -q https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j
import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


# Start a SparkSession
import findspark
findspark.init()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=1f4ecdcae0a3eff3f43496b3b6734a31d32b85c5b45edd2ff631e3f5fecf36d3
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, ignoreLeadingWhiteSpace=True)

# Show DataFrame
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [31]:
# Create a temporary view of the DataFrame.
df.createOrReplaceTempView('Homes_sales')


In [12]:
# Average price for a four bedroom house sold each year
spark.sql("SELECT ROUND(AVG(price), 2) as avg_price FROM Homes_sales WHERE bedrooms = 4").show()


+---------+
|avg_price|
+---------+
|299661.01|
+---------+



In [18]:
# Average price of a home for each year where the home was built that have 3 bedrooms and 3 bathrooms 
spark.sql("SELECT date_built as Year_built, ROUND(AVG(price), 2) as avg_price FROM Homes_sales WHERE bedrooms = 3  AND bathrooms = 3 GROUP BY date_built").show()



+----------+---------+
|Year_built|avg_price|
+----------+---------+
|      2016|290555.07|
|      2012|293683.19|
|      2017|292676.79|
|      2014|290852.27|
|      2013|295962.27|
|      2011|291117.47|
|      2015| 288770.3|
|      2010|292859.62|
+----------+---------+



In [19]:
# Average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,and are greater than or equal to 2,000 square feet 
spark.sql("SELECT date_built AS Year_Built, ROUND(AVG(price), 2) as avg_price\
          FROM Homes_sales\
          WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000\
          GROUP BY date_built").show()

+----------+---------+
|Year_Built|avg_price|
+----------+---------+
|      2016| 293965.1|
|      2012|307539.97|
|      2017|280317.58|
|      2014|298264.72|
|      2013|303676.79|
|      2011|276553.81|
|      2015|297609.97|
|      2010|285010.22|
+----------+---------+



In [21]:
# Rating for the average price of a home where the homes are greater thanor equal to $350,000

start_time = time.time()

spark.sql("SELECT view as View_Rating, ROUND(AVG(price), 2) as avg_price\
          FROM Homes_sales\
          WHERE price >= 350000\
          GROUP BY view").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------+----------+
|View_Rating| avg_price|
+-----------+----------+
|          7| 403005.77|
|         51| 788128.21|
|         15|  404673.3|
|         54| 798684.82|
|         11| 399548.12|
|         29| 397771.65|
|         69| 750537.94|
|         42|  396964.5|
|         87| 1072285.2|
|         73| 752861.18|
|         64| 767036.67|
|          3|  398867.6|
|         30|  397862.0|
|         34| 401419.75|
|         59|  791453.0|
|          8| 398592.71|
|         28| 402124.62|
|         22| 402022.68|
|         85|1056336.74|
|         16| 399586.53|
+-----------+----------+
only showing top 20 rows

--- 0.9798128604888916 seconds ---


In [35]:
# Caching the the temporary table home_sales.
spark.sql("cache table Homes_sales")

DataFrame[]

In [41]:
# Verifying if table was cached 
spark.catalog.isCached("Homes_sales")

True

In [42]:
# Filtering out the view ratings with average price greater than or equal to $350,000 
start_time = time.time()

spark.sql("SELECT view as View_Rating, ROUND(AVG(price), 2) as avg_price\
          FROM Homes_sales\
          WHERE price >= 350000\
          GROUP BY view").show()

print("--- %s seconds ---" % (time.time() - start_time))


+-----------+----------+
|View_Rating| avg_price|
+-----------+----------+
|          7| 403005.77|
|         51| 788128.21|
|         15|  404673.3|
|         54| 798684.82|
|         11| 399548.12|
|         29| 397771.65|
|         69| 750537.94|
|         42|  396964.5|
|         87| 1072285.2|
|         73| 752861.18|
|         64| 767036.67|
|          3|  398867.6|
|         30|  397862.0|
|         34| 401419.75|
|         59|  791453.0|
|          8| 398592.71|
|         28| 402124.62|
|         22| 402022.68|
|         85|1056336.74|
|         16| 399586.53|
+-----------+----------+
only showing top 20 rows

--- 0.5284924507141113 seconds ---


In [43]:
# Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("homes_sales_partitioned")

In [44]:
# Read the parquet formatted data
p_df_p=spark.read.parquet('homes_sales_partitioned')

In [45]:
# temporary table for the parquet data.
p_df_p.createOrReplaceTempView('homes_sales_p')

In [46]:
# filters out the view ratings with average price of greater than or equal to $350,000 with the parquet DataFrame.

start_time = time.time()

spark.sql("SELECT view as View_Rating, ROUND(AVG(price), 2) as avg_price\
          FROM homes_sales_p\
          WHERE price >= 350000\
          GROUP BY view").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------+----------+
|View_Rating| avg_price|
+-----------+----------+
|          7| 403005.77|
|         51| 788128.21|
|         15|  404673.3|
|         54| 798684.82|
|         11| 399548.12|
|         29| 397771.65|
|         69| 750537.94|
|         42|  396964.5|
|         73| 752861.18|
|         87| 1072285.2|
|         64| 767036.67|
|          3|  398867.6|
|         30|  397862.0|
|         34| 401419.75|
|         59|  791453.0|
|          8| 398592.71|
|         28| 402124.62|
|         22| 402022.68|
|         85|1056336.74|
|         35| 401934.21|
+-----------+----------+
only showing top 20 rows

--- 1.094820261001587 seconds ---


In [48]:
# Uncache the home_sales temporary table.
spark.sql("uncache table Homes_sales")

DataFrame[]

In [50]:
# 15. CVerifyinh if the temporary table was uncached 

spark.catalog.isCached("Homes_sales")

False